In [7]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import tqdm as notebook_tqdm
import numpy as np
from tqdm import tqdm

C:\Users\Hp\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#creating new document
dataset = fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'))

texts = dataset.data
labels = dataset.target
target_names = dataset.target_names

print(len(texts))
print(target_names[:5])

18846
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware']


In [9]:
#converting document into dataframe
df = pd.DataFrame({"doc_id": list(range(len(texts))),"text": texts,"label": labels})
df["label_name"] = df["label"].apply(lambda i: target_names[i])

In [10]:
#cleaning special character ,/n,/t
df.loc[:18845, "text"] = df.loc[:18845, "text"].str.replace(r"[\n\t]+", " ", regex=True)

In [11]:
#calling embeding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
texts = df["text"].tolist()

In [12]:
doc_embeddings = embedding_model.encode(texts,batch_size =32,show_progress_bar=True)
doc_embeddings =np.array(doc_embeddings,dtype="float32")

Batches: 100%|███████████████████████████████████████████████████████████████████████| 589/589 [09:52<00:00,  1.01s/it]


In [14]:
# Semantic Search

def search_docs(query, top_k=5):
    q_emb = embedding_model.encode([query])
    sims = cosine_similarity(q_emb, doc_embeddings).flatten()
    
    top_idx = sims.argsort()[::-1][:top_k]
    
    result = []
    for idx in top_idx:
        doc_id = df.iloc[idx]["doc_id"]
        score = sims[idx]
        text = df.iloc[idx]["text"][:300]  # show preview
        result.append((doc_id, score, text))
    
    return result

In [15]:
#search_docs("computer graphics", top_k=3)

In [16]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [17]:
def summarize_with_local_llm(query, retrieved_text, length="medium"):
    
    if length == "short":
        max_len = 70
        min_len = 30
    elif length == "long":
        max_len = 200
        min_len = 80
    else:
        max_len = 120
        min_len = 50

    chunk = retrieved_text[:4000]

    summary = summarizer(chunk, max_length=max_len, min_length=min_len, do_sample=False)

    return summary[0]['summary_text']

In [18]:
#Build final pipeline: Search → Summarize
def answer_query(query, top_k=5, length="medium"):
    results = search_docs(query, top_k)
    
    combined_text = ""
    for doc_id, score, preview in results:
        full_text = df[df["doc_id"] == doc_id]["text"].values[0]
        combined_text += full_text + "\n\n"

    summary = summarize_with_local_llm(query, combined_text, length)
    
    return results, summary

In [19]:
query = "Explain discussions about religion"
results, summary = answer_query(query, top_k=5, length="medium")

print("===== SUMMARY =====")
print(summary)

print("\n===== TOP DOCUMENTS =====")
for doc_id, score, preview in results:
    print(f"Doc ID: {doc_id} | Score: {score:.4f}")
    print(preview)
    print("---")

===== SUMMARY =====
Religion is a drug... i believe you can discuss religion. religion is personal belief system. people loudly proclaiming their beliefs are crossing the border from religion to Religion. Religion is something that is contracted from others and does not reflect the persons value judgements (other than perhaps "i think i'll be brainwashed today").

===== TOP DOCUMENTS =====
Doc ID: 11974 | Score: 0.5912
 no its not. its due to the fact that there are two issues here: Religion and religion. religion is personal belief system. Religion is a memetic virus. people loudly proclaiming their beliefs are crossing the border from religion -> Religion.  people that want to "save" others are firmly entrenched
---
Doc ID: 10538 | Score: 0.5269
 I hope I didn't award custody, Rich.  I purposely used "handle" in order to  avoid doing so - i.e., that happens to be what religions do (of course there are aberrations like "scientific" creationism).  I used "best" in part to indicate  tha

In [20]:
#acuracy check 

def hit_at_k(query, expected_label, k=5):
    results = search_docs(query, top_k=k)

    for doc_id, score, preview in results:
        label = df[df["doc_id"] == doc_id]["label_name"].values[0]
        if label == expected_label:
            return 1
    return 0

def calculate_retrieval_accuracy(sample_size=50, k=5):
    hits = 0

    samples = df.sample(sample_size, random_state=42)

    for _, row in samples.iterrows():
        query = row["label_name"]      # use label as query
        expected_label = row["label_name"]

        hits += hit_at_k(query, expected_label, k)

    accuracy = hits / sample_size
    return accuracy

In [21]:
accuracy = calculate_retrieval_accuracy(sample_size=50, k=5)
print(f"Hit@5 Retrieval Accuracy: {accuracy:.2f}")

Hit@5 Retrieval Accuracy: 0.96
